# Livestock Analysis

Livestock analysis based on the dataframe previously created using scrape routine and OCR to extract data from auctions pdf's from local auction website.

### Import Modules

In [288]:
# Pandas
import pandas as pd
# Numpy
import numpy as np
# Import math for ceil method in calculating bin
import math
# Plotly
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True) # Offline plotting
# Plotly controls
from IPython.display import display, HTML

### Clean Up Dataframe

In [4]:
# Read dataframe
df = pd.read_csv("data.csv")
# Remove useless columns
df.drop(["Unnamed: 0", "age"], axis=1, inplace=True)
# Turn date type from objecto to datetime
df["date"] = df["date"].astype('datetime64[ns]')
# Create sex column
df["sex"] = df.category.str.contains("MACHO").replace({True: 1, False: 0})
# Fill NaN values with 0 = Female. Checked manually. The ones missing are in file 17-08-31 and are all female 1st iron.
df["sex"].fillna(0, inplace=True)
# Create iron column
df["iron"] = df.category.str.contains("PRIMERA") + df.category.str.contains("1a")
# Fill NaN values with 0 = Female. Checked manually. The ones missing are in file 17-08-31 and are all female 1st iron.
df["iron"].fillna(1, inplace=True)
df["iron"] = df["iron"].replace({0: 2})
# Change types
df["sex"] = df["sex"].astype(np.int32)
df["iron"] = df["iron"].astype(np.int32)
# Drop category column
df.drop(["category"], axis=1, inplace=True)
# Check if there are no null values in sex column and in iron column
if (df["sex"].isnull().values.any() == False) and (df["iron"].isnull().values.any() == False): print("Ente gut, alles gut!")

Ente gut, alles gut!


### Sales Volumes By Units Sold and COP

Histogram showing the volumes transacted according to the weight. In this specific example we are interested in the males. Segmented by numer of units sold and amount of money transacted in the local currency: COP.

In [287]:
# Dataframe of Males: ML & MC types.
df_1 = df[(df["type"] == "ML") | (df["type"] == "MC")]
df_1["year"] = df["date"].apply(lambda x: x.year)

/Users/pedroatencio/anaconda/envs/py361/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [264]:
def volume_hist_by_weight(dataframe):
    # Histogram dict
    weight_hist_dict = {}
    # Set bin size for weight
    bin_size = 10
    # Main routine
    for index, row in dataframe.iterrows():
        # Calculate weight bin
        weight_bin = int(bin_size*math.ceil(row["weight"]/bin_size) - (bin_size/2))
        # Update weight_hist_dict
        if weight_bin in weight_hist_dict:
            weight_hist_dict[weight_bin][0] += row["quantity"]
            weight_hist_dict[weight_bin][1] += row["quantity"]*row["price_unit"]
        else:
            weight_hist_dict[weight_bin] = [row["quantity"], row["quantity"]*row["price_unit"]]
    # Sorted keys
    sorted_keys = sorted([k for k in weight_hist_dict])
    # Data dictionary for output
    data_dict = {"unit_volume": [], "cop_volume": [], "bins": []}
    # Populate data dictionary
    for key in sorted_keys:
        data_dict["bins"].append(key)
        data_dict["unit_volume"].append(weight_hist_dict[key][0])
        data_dict["cop_volume"].append(weight_hist_dict[key][1]) 
    # Return data dict
    return data_dict

In [298]:
sum(volume_hist_by_weight(df_1[df_1["year"]==2016])["cop_volume"])

52599422624

In [299]:
def plot_volumes(volume_type):
    # Define years
    years = ['2013', '2014', '2015', '2016', '2017']

    # Make figure
    figure = {
        'data': [],
        'layout': {},
        'frames': []
    }

    # Fill in most of layout
    figure['layout']['xaxis'] = {'range': [0, 700], 'title': 'Weight'}
    figure['layout']['yaxis'] = {'title': 'Volume [Units]'}
    figure['layout']['hovermode'] = 'closest'
    figure['layout']['sliders'] = {
        'args': [
            'transition', {
                'duration': 1000,
                'easing': 'cubic-in-out'
            }
        ],
        'initialValue': '2013',
        'plotlycommand': 'animate',
        'values': years,
        'visible': True
    }
    figure['layout']['updatemenus'] = [
        {
            'buttons': [
                {
                    'args': [None, {'frame': {'duration': 5000, 'redraw': True},
                             'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                    'label': 'Play',
                    'method': 'animate'
                },
                {
                    'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate',
                    'transition': {'duration': 0}}],
                    'label': 'Pause',
                    'method': 'animate'
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }
    ]

    sliders_dict = {
        'active': 0,
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Year:',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': []
    }

    # Make data
    year = 2013

    # Initialize data
    df_1_year = df_1[df_1['year'] == year]
    data = volume_hist_by_weight(df_1_year)
    data_dict = go.Bar(
        x=data["bins"],
        y=data[volume_type]
    )
    figure['data'].append(data_dict)

    # Make frames
    for year in years:
        frame = {'data': [], 'name': str(year)}

        df_1_year = df_1[df_1['year'] == int(year)]
        data = volume_hist_by_weight(df_1_year)

        data_dict = go.Bar(
            x=data["bins"],
            y=data[volume_type]
        )
        frame['data'].append(data_dict)

        figure['frames'].append(frame)
        slider_step = {'args': [
            [year],
            {'frame': {'duration': 500, 'redraw': True},
             'mode': 'immediate',
           'transition': {'duration': 500}}
         ],
         'label': year,
         'method': 'animate'}
        sliders_dict['steps'].append(slider_step)

    figure['layout']['sliders'] = [sliders_dict]

    iplot(figure)

#### Sales Volume in Units

In [285]:
# Plot unit volume
plot_volumes("unit_volume")

#### Sales Volume in COP

In [286]:
# Plot COP volume
plot_volumes("cop_volume")

### What We Care About: Profit

The point is to find the parameters that give the optimal profit. There are 2 independent variables: the initial weight of the animal and the number of months is kept - this is assuming a fixed number of kilograms per day increase. 

Let:

1. $p_i$ be the purchase price.
2. $p_f$ be the selling price.
3. $\overline{kg}_i$ be the average kilograms per animal on purchase.
4. $\overline{kg}_f$ be the average kilograms per animal when selling.
5. $\Delta t$ be the time lapse in months.
6. $n$ be the number of animals bought.

Then the $ROI$ will be:

$$ROI = \frac{n\cdot (p_f\cdot \overline{kg}_f-p_i\cdot \overline{kg}_i)}{n\cdot p_i\cdot \overline{kg}_i} 
= \frac{p_f\cdot \overline{kg}_f}{p_i\cdot \overline{kg}_i}-1$$

Now, given that $\overline{kg}_f$ depends on the purchase weight and the number of months the livestock is fed, we have:

$$ROI = \frac{p_f\cdot (\overline{kg}_i+ \dot{\overline{kg}}\cdot \Delta t)}{p_i\cdot \overline{kg}_i}-1 = \frac{p_f}{p_i}\cdot \left(\frac{\dot{\overline{kg}}\cdot \Delta t}{\overline{kg}_i} +1\right)-1$$

The former equation shows the $ROI$ of the operation, where: $\dot{\overline{kg}}$ is the average weight gain per month per unit. However, this equation depends on the time lapse - number of months - of the operation. In order to have a standard measure lets look at the annualized $ROI$.

$$ROI_{ANNUALIZED} = \left[\frac{p_f}{p_i}\cdot \left(\frac{\dot{\overline{kg}}\cdot \Delta t}{\overline{kg}_i} +1\right)-1\right]^{\left(\frac{12}{\Delta t}\right)}$$